In [20]:
import pandas as pd
from pathlib import Path

results_dir = Path.cwd().parent.parent / "data" / "results"

lo_raw = pd.read_csv(results_dir / "logistic_regression_raw_metrics.csv", index_col=0)
lo_processed = pd.read_csv(results_dir / "logistic_regression_processed_metrics.csv", index_col=0)

xgb_raw = pd.read_csv(results_dir / "xgboost_processed_raw.csv", index_col=0)
xgb_processed = pd.read_csv(results_dir / "xgboost_variants_metrics.csv", index_col=0)
row_0_5 = xgb_processed.query("variant == 'default_0.5'").iloc[0]
row_Recall = xgb_processed.query("variant == 'Recall_opt'").iloc[0]
row_F2 = xgb_processed.query("variant == 'F2_opt'").iloc[0]



# lightgbm_raw = pd.read_csv(results_dir / "lightgbm_raw_metrics.csv", index_col=0)
lightgbm_processed = pd.read_csv(results_dir / "confusion_matrix_lightgbm.csv", index_col=0)
# Extract confusion matrix values
TN = lightgbm_processed.loc["Actual_0", "Pred_0"]
FP = lightgbm_processed.loc["Actual_0", "Pred_1"]
FN = lightgbm_processed.loc["Actual_1", "Pred_0"]
TP = lightgbm_processed.loc["Actual_1", "Pred_1"]

# Compute metrics
precision_lgb = TP / (TP + FP)
recall_lgb = TP / (TP + FN)
f1_lgb = 2 * precision_lgb * recall_lgb / (precision_lgb + recall_lgb)
accuracy_lgb = (TP + TN) / (TP + TN + FP + FN)

# Nếu bạn có time ở file khác thì đặt vào, còn không thì 0
time_lgb = 0,87565


metrics = ["precision", "recall", "f1-score", "accuracy", "time_seconds"]

# Tạo dict để tránh lỗi transpose
data = {
    "Raw Logistic": [
        lo_raw.loc[m, "weighted avg"] if m in ["precision","recall","f1-score"] 
        else lo_raw.loc[m, lo_raw.columns[0]]
        for m in metrics
    ],
    "Processed Logistic": [
        lo_processed.loc[m, "weighted avg"] if m in ["precision","recall","f1-score"] 
        else lo_processed.loc[m, lo_processed.columns[0]]
        for m in metrics
    ],
    "Raw XGBoost": [
        xgb_raw.loc[m, "weighted avg"] if m in ["precision","recall","f1-score"] 
        else xgb_raw.loc[m, xgb_raw.columns[0]]
        for m in metrics
    ],
    "Processed XGBoost": [
        row_0_5[m] for m in metrics
    ],
    "Processsed XGBoost (Recall_opt)": [
        row_Recall[m] for m in metrics
    ],
    "Processsed XGBoost (F2_opt)": [
        row_F2[m] for m in metrics
    ],
    
    # "Raw LightGBM": [
    #     lightgbm_raw.loc[m, "weighted avg"] if m in ["precision","recall","f1-score"] 
    #     else lightgbm_raw.loc[m, lightgbm_raw.columns[0]]
    #     for m in metrics
    # ],
    "Processed LightGBM": [
    precision_lgb,
    recall_lgb,
    f1_lgb,
    accuracy_lgb,
    time_lgb,
    ],

    #     lightgbm_processed.loc[m, "weighted avg"] if m in ["precision","recall","f1-score"] 
    #     else lightgbm_processed.loc[m, lightgbm_processed.columns[0]]
    #     for m in metrics
    # ],
}

# Metric là index, không phải cột
comparison = pd.DataFrame(data, index=metrics)

# Chuyển vị
comparison_T = comparison.transpose()

# Lưu file không index
# comparison_T.to_csv(results_dir / "logistic_comparison.csv", index=False)
comparison_T.to_csv(results_dir / "model_comparison.csv", index=True)

comparison_T.head()


,precision,recall,f1-score,accuracy,time_seconds
Raw Logistic,0.89497,0.946029,0.919791,0.946029,0.079824
Processed Logistic,0.94534,0.685294,0.774441,0.685294,0.059329
Raw XGBoost,0.894866,0.943992,0.918773,0.943992,0.802062
Processed XGBoost,0.333333,0.1,0.153846,0.946078,0.931239
Processsed XGBoost (Recall_opt),0.105134,0.86,0.187364,0.634314,1.263573
